# SALES ANALYSIS

In [48]:
# Start with loading all necessary libraries 
import pandas as pd
import numpy as np
import glob
import os
import re
from itertools import combinations
from collections import Counter

# Viz libraries 
import plotly.express as px  
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

### Merging 12 months data in single dataset

In [2]:
os.chdir(r'https://github.com/KyrilFionov/Python-Portfolio/tree/main/Sales-Analysis/dataset')

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#Combine all files in the list
combined_df = pd.concat([pd.read_csv(f) for f in all_filenames ])

In [3]:
# Brief check of data
df = combined_df
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"


### Treating NULL values

In [4]:
# Identify NULL values
# Option 1
def num_missing(x):
  return sum(x.isnull())

#Applying per column:
print ("Missing values per column:")
print (df.apply(num_missing, axis=0))

Missing values per column:
Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64


In [5]:
# Identify NULL values
# Option 2
df.apply(lambda x: sum(x.isnull()), axis = 0)

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [6]:
# Remove NULL values
df.dropna(how='all', inplace = True)
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"


In [7]:
# Validate NULL values removal
df.apply(lambda x: sum(x.isnull()), axis = 0)

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64

In [8]:
df.describe()
# We can see that CONCAT did not remove column headers from the rows

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
count,186305,186305,186305,186305,186305,186305
unique,178438,20,10,24,142396,140788
top,Order ID,USB-C Charging Cable,1,11.95,Order Date,Purchase Address
freq,355,21903,168552,21903,355,355


### Remove header rows from the dataset

In [9]:
temp_df = df.loc[df['Order Date'].str.contains("Order Date", case=False)]
temp_df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
254,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
705,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1101,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
2875,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
3708,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
...,...,...,...,...,...,...
10443,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
10784,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
10813,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
11047,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


In [10]:
df = df.drop(df.loc[df['Order Date'].str.contains("Order Date", case=False)].index)
df.reset_index()

,index,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
3,3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"
...,...,...,...,...,...,...,...
182730,13617,222905,AAA Batteries (4-pack),1,2.99,06/07/19 19:02,"795 Pine St, Boston, MA 02215"
182731,13618,222906,27in FHD Monitor,1,149.99,06/01/19 19:29,"495 North St, New York City, NY 10001"
182732,13619,222907,USB-C Charging Cable,1,11.95,06/22/19 18:57,"319 Ridge St, San Francisco, CA 94016"
182733,13620,222908,USB-C Charging Cable,1,11.95,06/26/19 18:35,"916 Main St, San Francisco, CA 94016"


In [11]:
check = df.loc[df['Order Date'].str.contains("Order Date", case=False)]
check

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


### Check data types before further processing

In [12]:
df.dtypes

Order ID            object
Product             object
Quantity Ordered    object
Price Each          object
Order Date          object
Purchase Address    object
dtype: object

#### Convert object (string) data to correct formats

In [13]:
df["Quantity Ordered"] = pd.to_numeric(df["Quantity Ordered"])
df["Price Each"] = pd.to_numeric(df["Price Each"])
df["Order Date"] = pd.to_datetime(df["Order Date"])

In [14]:
df.describe()

,Quantity Ordered,Price Each
count,182735.000000,182735.000000
mean,1.124333,184.315392
std,0.442937,332.684344
min,1.000000,2.990000
25%,1.000000,11.950000
50%,1.000000,14.950000
75%,1.000000,150.000000
max,9.000000,1700.000000


### Taks 1: What was the best month for sales? How much was earned that month?

In [15]:
# First I have to create month values
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month
df['Year-Month'] = df['Order Date'].dt.strftime('%Y-%m')
df['Revenue'] = round(df['Quantity Ordered']*df['Price Each'],2)

df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Year,Month,Year-Month,Revenue
0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",2019,12,2019-12,1700.00
1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",2019,12,2019-12,600.00
2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",2019,12,2019-12,11.95
3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",2019,12,2019-12,149.99
4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",2019,12,2019-12,11.95


In [80]:
reporting_year = df['Year'].max()-1

monthly_revenue_df = df.groupby(['Year-Month','Year','Month'])['Revenue'].sum()
monthly_revenue_df = monthly_revenue_df.reset_index()
monthly_revenue_df = monthly_revenue_df.loc[monthly_revenue_df['Year']==reporting_year]
monthly_revenue_df['Yearly AVG Revenue'] = round(np.mean(monthly_revenue_df['Revenue']),2)
monthly_revenue_df['Monthly Revenue vs Yearly AVG'] = monthly_revenue_df['Revenue']-monthly_revenue_df['Yearly AVG Revenue']

monthly_revenue_df

,Year-Month,Year,Month,Revenue,Yearly AVG Revenue,Monthly Revenue vs Yearly AVG
0,2019-01,2019,1,1777841.00,2822592.46,-1044751.46
1,2019-02,2019,2,2158127.48,2822592.46,-664464.98
2,2019-03,2019,3,2755969.40,2822592.46,-66623.06
3,2019-04,2019,4,3336376.42,2822592.46,513783.96
4,2019-05,2019,5,3101881.04,2822592.46,279288.58
5,2019-06,2019,6,2524464.99,2822592.46,-298127.47
6,2019-07,2019,7,2587444.91,2822592.46,-235147.55
7,2019-08,2019,8,2191698.31,2822592.46,-630894.15
8,2019-09,2019,9,2050361.26,2822592.46,-772231.20
9,2019-10,2019,10,3679254.16,2822592.46,856661.70


#### Visualize with Plotly

In [162]:
# Set styling parameters
colors = {
    'background': '#F2F2F2'
    ,'text': '#333333'
    ,'colorscale': 'Sunsetdark'
}

main_trace = go.Scatter(
    x=monthly_revenue_df['Year-Month']
    ,y=monthly_revenue_df['Revenue']
    ,name = 'Revenue'
    ,mode = 'lines'
    ,customdata=monthly_revenue_df['Monthly Revenue vs Yearly AVG']
    ,hovertemplate='Month: %{x}<br>Revenue: $%{y:.2s}<br>vs Yearly AVG: %{customdata:$,.2s}<extra></extra>'
)

# Create the line chart
avg_trace = go.Scatter(
    x=monthly_revenue_df['Year-Month']
    ,y=monthly_revenue_df['Yearly AVG Revenue']
    ,name = 'Yearly AVG Revenue'
    ,mode = 'lines'
    ,hovertemplate='Yearly AVG Revenue: $%{y:.2s} <extra></extra>'
)
data = [main_trace,avg_trace]

# Set the layout
layout = go.Layout(
    title=f'{reporting_year} Revenue by Month'
    ,xaxis=dict(
        title='Order Date'
        ,showgrid = False
        ,tickmode='array'
        ,tickformat='%b-%y'
        ,tickvals=monthly_revenue_df['Year-Month']
    )
    ,yaxis=dict(
        title='Revenue'
        ,showgrid=False
        ,tickformat='$.1s'
        ,zeroline=False
    )
    ,plot_bgcolor=colors['background']
    ,paper_bgcolor=colors['background']
)

# Create the figure and display it
revenue_linechart = go.Figure(data, layout)
revenue_linechart.show()


### Task 2: What city had the biggest sales?

In [18]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Year,Month,Year-Month,Revenue
0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",2019,12,2019-12,1700.00
1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",2019,12,2019-12,600.00
2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",2019,12,2019-12,11.95
3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",2019,12,2019-12,149.99
4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",2019,12,2019-12,11.95


In [19]:
# First I need to extract City name from the address
# As it often happens with geographies - there can be the same city name within multiple countries (or states in our case)
# That is why I extract both city and a state to avoid incorect assignments
def get_city(address):
    return address.split(',')[1]

def get_state(address):
    return address.split(',')[2].split(' ')[1]
df['City'] = df['Purchase Address'].apply(lambda x: get_city(x) + ' (' + get_state(x) + ')')

In [20]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Year,Month,Year-Month,Revenue,City
0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",2019,12,2019-12,1700.00,New York City (NY)
1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",2019,12,2019-12,600.00,New York City (NY)
2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",2019,12,2019-12,11.95,New York City (NY)
3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",2019,12,2019-12,149.99,San Francisco (CA)
4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",2019,12,2019-12,11.95,Atlanta (GA)


In [107]:
top_revenue_city_df = df.groupby(['Year','City'])[['Revenue']].sum().sort_values('Revenue',ascending = True)
top_revenue_city_df = top_revenue_city_df.reset_index()
top_revenue_city_df = top_revenue_city_df.loc[top_revenue_city_df['Year']==reporting_year]

top_revenue_city_df['Highest Revenue'] = top_revenue_city_df['Revenue'].max(numeric_only=True)
top_revenue_city_df['vs Leader'] = top_revenue_city_df['Highest Revenue'] - top_revenue_city_df['Revenue']
best_performing_city = top_revenue_city_df.loc[top_revenue_city_df['Revenue'].nlargest(1).index[0], 'City']

top_revenue_city_df

,Year,City,Revenue,Highest Revenue,vs Leader
8,2019,Portland (ME),444110.53,8121636.06,7677525.53
9,2019,Austin (TX),1786245.54,8121636.06,6335390.52
10,2019,Portland (OR),1832385.72,8121636.06,6289250.34
11,2019,Seattle (WA),2692898.61,8121636.06,5428737.45
12,2019,Dallas (TX),2716078.77,8121636.06,5405557.29
13,2019,Atlanta (GA),2741642.05,8121636.06,5379994.01
14,2019,Boston (MA),3602753.97,8121636.06,4518882.09
15,2019,New York City (NY),4580318.29,8121636.06,3541317.77
16,2019,Los Angeles (CA),5353039.94,8121636.06,2768596.12
17,2019,San Francisco (CA),8121636.06,8121636.06,0.00


#### Visualize with Plotly

In [116]:
# Set styling parameters
colors = {
    'background': '#F2F2F2'
    ,'text': '#333333'
    ,'colorscale': 'Sunsetdark'
}
# Create the bar chart
data = go.Bar(
    x=top_revenue_city_df['Revenue']
    ,y=top_revenue_city_df['City']
    ,orientation='h'
    ,marker=dict(
        color=top_revenue_city_df['Revenue']
        ,showscale=True
        ,colorscale=colors['colorscale']
        ,line=dict(color=colors['background'])
    )
    ,customdata=top_revenue_city_df['vs Leader']
    ,hovertemplate = f"City: %{{y}}<br>Revenue: $%{{x:.2s}}<br>vs{best_performing_city}: $%{{customdata:.2s}} <extra></extra>")

# Set the layout
layout = go.Layout(
    title=f'{reporting_year} Revenue by City'
    ,xaxis=dict(
        title='Revenue ($)'
        ,showgrid=False
        ,zeroline=False
    )
    ,yaxis=dict(
        title='City'
        ,showgrid=False
        ,zeroline=False
        ,categoryorder='total ascending'
    )
    ,plot_bgcolor=colors['background']
    ,paper_bgcolor=colors['background']
)

# Create the figure and display it
top_revenue_city_chart = go.Figure(data=data, layout=layout)
top_revenue_city_chart.show()


### Task 3: What products are most frequently sold together?

In [46]:
# step 1 - Identify Order IDs with multiple products

multi_unit_orders_df = df[df['Order ID'].duplicated(keep = False)]
multi_unit_orders_df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Year,Month,Year-Month,Revenue,City
16,295681,Google Phone,1,600.00,2019-12-25 12:37:00,"79 Elm St, Boston, MA 02215",2019,12,2019-12,600.00,Boston (MA)
17,295681,USB-C Charging Cable,1,11.95,2019-12-25 12:37:00,"79 Elm St, Boston, MA 02215",2019,12,2019-12,11.95,Boston (MA)
18,295681,Bose SoundSport Headphones,1,99.99,2019-12-25 12:37:00,"79 Elm St, Boston, MA 02215",2019,12,2019-12,99.99,Boston (MA)
19,295681,Wired Headphones,1,11.99,2019-12-25 12:37:00,"79 Elm St, Boston, MA 02215",2019,12,2019-12,11.99,Boston (MA)
36,295698,Vareebadd Phone,1,400.00,2019-12-13 14:32:00,"175 1st St, New York City, NY 10001",2019,12,2019-12,400.00,New York City (NY)


In [47]:
# step 2 - Bundle (agregate) products from multiple rows into single column

multi_unit_orders_df = multi_unit_orders_df.groupby('Order ID')['Product'].apply(lambda x: x.str.cat(sep=',')).reset_index()
multi_unit_orders_df.head()

,Order ID,Product
0,141275,"USB-C Charging Cable,Wired Headphones"
1,141290,"Apple Airpods Headphones,AA Batteries (4-pack)"
2,141365,"Vareebadd Phone,Wired Headphones"
3,141384,"Google Phone,USB-C Charging Cable"
4,141450,"Google Phone,Bose SoundSport Headphones"


In [55]:
# step 3 - count repeatable items

number_of_bundled_units = 2
show_x_combinations = 10

count = Counter()
for row in multi_unit_orders_df['Product']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list,number_of_bundled_units)))

most_common_products = count.most_common(show_x_combinations)

most_common_products = pd.DataFrame(most_common_products,columns = ['Bundled Products','Frequency'])
most_common_products

,Bundled Products,Frequency
0,"(iPhone, Lightning Charging Cable)",973
1,"(Google Phone, USB-C Charging Cable)",954
2,"(iPhone, Wired Headphones)",437
3,"(Google Phone, Wired Headphones)",401
4,"(Vareebadd Phone, USB-C Charging Cable)",348
5,"(iPhone, Apple Airpods Headphones)",343
6,"(Google Phone, Bose SoundSport Headphones)",219
7,"(USB-C Charging Cable, Wired Headphones)",156
8,"(Vareebadd Phone, Wired Headphones)",141
9,"(Lightning Charging Cable, Wired Headphones)",92


### Task 4: What time should we display advertisemens to maximize the likelihood of customer’s buying product?

In [143]:
# First I create a Day-of-the-week and Hour columns
df['Order Weekday'] = df['Order Date'].dt.strftime('%a')
df['Order Hour'] = df['Order Date'].dt.strftime('%H:00')
df.head()


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Year,Month,Year-Month,Revenue,City,Order Hour,Order Weekday
0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",2019,12,2019-12,1700.00,New York City (NY),00:00,Mon
1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",2019,12,2019-12,600.00,New York City (NY),07:00,Sun
2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",2019,12,2019-12,11.95,New York City (NY),18:00,Thu
3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",2019,12,2019-12,149.99,San Francisco (CA),15:00,Sun
4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",2019,12,2019-12,11.95,Atlanta (GA),12:00,Wed


In [147]:
marketing_df = df.groupby(['Order Weekday','Order Hour']).agg({'Order ID':'nunique','Revenue':'sum'}).reset_index()
marketing_df = marketing_df.rename(columns={'Order ID': 'Number of Orders'})

marketing_df


,Order Weekday,Order Hour,Number of Orders,Revenue
0,Fri,00:00,566,111153.23
1,Fri,01:00,315,59418.22
2,Fri,02:00,166,35738.80
3,Fri,03:00,94,26355.34
4,Fri,04:00,101,22673.33
...,...,...,...,...
163,Wed,19:00,1739,336868.41
164,Wed,20:00,1531,318992.57
165,Wed,21:00,1499,302088.35
166,Wed,22:00,1201,238376.32


#### Visualize with Plotly

In [161]:
# Set styling parameters
colors = {
    'background': '#F2F2F2'
    ,'text': '#333333'
    ,'colorscale': 'Sunsetdark'
}

# Create heatmap traces
revenue_trace = go.Heatmap(
    x=marketing_df['Order Weekday']
    ,y=marketing_df['Order Hour']
    ,z=marketing_df['Revenue'].values.tolist()
    ,colorscale=colors['colorscale']
    ,showscale=False
    ,hovertemplate = 'Day: %{x}<br>Time: %{y}<br>Revenue: $%{z:.2s}<extra></extra>'

)
orders_trace = go.Heatmap(
    x=marketing_df['Order Weekday']
    ,y=marketing_df['Order Hour']
    ,z=marketing_df['Number of Orders'].values.tolist()
    ,colorscale=colors['colorscale']
    ,showscale=False
    ,hovertemplate = 'Day: %{x}<br>Time: %{y}<br># of Orders: %{z}<extra></extra>'

)
data = [revenue_trace,orders_trace]

#Creating subplots
marketing_heatmap = make_subplots(
    rows=1
    ,cols=2
    ,subplot_titles=['Revenue', '# of Orders']
)

marketing_heatmap.append_trace(revenue_trace, 1, 1)
marketing_heatmap.append_trace(orders_trace, 1, 2)

# Set the layout
layout = go.Layout(
    title=f'{reporting_year} Revenue by City'
    ,xaxis=dict(
        title='Revenue ($)'
        ,showgrid=False
        ,zeroline=False
    )
    ,plot_bgcolor=colors['background']
    ,paper_bgcolor=colors['background']
)

marketing_heatmap.update_layout(
    title=f'{reporting_year} Customers purchase distribution'
)

marketing_heatmap.update_xaxes(
    categoryorder = 'array'
    ,categoryarray = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
    ,tickmode = 'array'
)

# Create the figure and display it
marketing_heatmap.show()